<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Examples/blob/master/notebooks/nsclc-radiomics/src/nsclc_radiomics_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Exploring GCS-Buckets and Google Colab Integration </center>

## Environment Set Up and Data Download

### Environment Set Up

N.B. To access a free GPU on Colab:
`Edit > Notebooks Settings`

From the dropdown menu under `Hardware accelerator`, select `GPU`.

---
Check whether the filesystem was wiped or not:

In [ ]:
!ls

sample_data


Check whether the instance is a GPU instance.

This could be useful **for debug purposes** - e.g., to drive some operations in the following blocks (e.g., the install of the right version of Keras and import of TensorFlow).

In [ ]:
gpu_list = !nvidia-smi --list-gpus

has_gpu = False if "failed" in gpu_list[0] else True

has_gpu

False

---
<font size="+2" color="orange"><b>Be aware of Tensorflow/Keras compatibility problems.</b></font>

Ahmed's model was compiled and saved in such a way that only old versions of Tensorflow supports it.

Output of `pip3 freeze | grep tensor` on the IDC VM:
```
tensorflow-datasets==1.2.0
tensorflow-estimator==1.15.1
tensorflow-gpu==1.15.2
tensorflow-hub==0.6.0
tensorflow-io==0.8.1
tensorflow-metadata==0.21.1
tensorflow-probability==0.8.0
tensorflow-serving-api-gpu==1.15.0
```

For `pip3 freeze | grep Keras`:
```
Keras==1.2.2
Keras-Applications==1.0.8
Keras-Preprocessing==1.1.0
```

While running `nvcc --version` gets us:
```
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
```

<br>

Google Colab's GPU instances come with CUDA 10.1 pre-installed:

```
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
```

And, by default, newer versions of Tensorflow are installed (in which Keras is fully integrated in TF):

```
keras-vis==0.4.1

tensorflow==2.3.0
tensorflow-addons==0.8.3
tensorflow-datasets==2.1.0
tensorflow-estimator==2.3.0
tensorflow-gcs-config==2.3.0
tensorflow-hub==0.9.0
tensorflow-metadata==0.24.0
tensorflow-privacy==0.2.2
tensorflow-probability==0.11.0
```

<br>

Luckily, [it is possible to use the](https://colab.research.google.com/notebooks/tensorflow_version.ipynb) `%tensorflow_version 1.x` [magic to switch to an older version of TensorFlow](https://colab.research.google.com/notebooks/tensorflow_version.ipynb) (`1.15.2`). The magic command must be run before importing Tensorflow. Still, CUDA 10.1 is used (which probably means Tensorflow 1.15, not "compatible by default" with 10.1, was compiled from source)


We then will need to install an old version of Keras for/by which Ahmed's model was formatted.

---

In [ ]:
# FIXME: useful for debug purposes
if has_gpu:
  !pip3 install --force-reinstall Keras==1.2.2

In [ ]:
# FIXME: parse from a GitHub repository
requirements_list = ["numpy",
                     "pandas",
                     "SimpleITK",
                     "pydicom",
                     "matplotlib",
                     "seaborn"]

with open('requirements.txt', 'w') as fp:
  for item in requirements_list:
    fp.write("%s\n"%(item))

!pip3 install -r requirements.txt

     |████████████████████████████████| 44.9MB 85kB/s 
     |████████████████████████████████| 35.5MB 121kB/s 


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import SimpleITK as sitk

print("Python version      : ", sys.version.split('\n')[0])
print("Numpy version       : ", np.__version__)

# FIXME: useful for debug purposes
if has_gpu:
  %tensorflow_version 1.x
  import tensorflow as tf
  import keras
  print("TensorFlow version  : ", tf.__version__)
  print("Keras version       : ", keras.__version__)
  print("\nThis Colab instance is equipped with a GPU.")
else:
  print("\nThis Colab instance IS NOT equipped with a GPU.")


# ----------------------------------------

#everything that has to do with plotting goes here below
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## ----------------------------------------

# create new colormap appending the alpha channel to the selected one
# (so that we don't get a \"color overlay\" when plotting the segmask superimposed to the CT)
cmap = plt.cm.Reds
my_reds = cmap(np.arange(cmap.N))
my_reds[:,-1] = np.linspace(0, 1, cmap.N)
my_reds = ListedColormap(my_reds)

cmap = plt.cm.jet
my_jet = cmap(np.arange(cmap.N))
my_jet[:,-1] = np.linspace(0, 1, cmap.N)
my_jet = ListedColormap(my_jet)

## ----------------------------------------

import seaborn as sns

Python version      :  3.6.9 (default, Jul 17 2020, 12:50:27) 
Numpy version       :  1.18.5

This Colab instance IS NOT equipped with a GPU.


---

### Data Download

[How to Mount GCS bucket on Colab:](https://gist.github.com/korakot/f3600576720206363c734eca5f302e38)

First step is the authentication:

In [ ]:
from google.colab import auth
auth.authenticate_user()

Secondly, in order to run python functions while without a GitHub repository under IDC still, mount Google Drive's filesystem:

In [ ]:
from google.colab import drive

mount_point = '/mnt/gdrive'

if os.path.exists(mount_point):
  drive.flush_and_unmount()

drive.mount(mount_point)

Mounted at /mnt/gdrive


---

#### Create a Dummy Manifesto for TCIA's NSCLC-radiomics.

Let's start with `n_patients` patients:

In [68]:
n_patients = 10

gsutil_output = !gsutil -u idc-sandbox-000 ls gs://idc-tcia-1-nsclc-radiomics/dicom | head -n $n_patients

# get rid ot the rows in excess
gsutil_output = gsutil_output[:n_patients]
gsutil_output

['gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.100645791623978861750505647044382842552/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.100855571832074152951605738408734618579/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.101385144304667221218962368127740145808/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.10144581065043339122451692100395007780/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102719233427745502116416854281613435360/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102758679895916452229843467594324423877/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102978961983855858543476361768905978287/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.10331314816593639915006005461010274587/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.103863628921590543632414419591397821320/',
 'gs://idc-tcia-1-nsclc-radiomics/dicom

In [ ]:
# populate a dataframe starting from the gsutil command output
manifesto_dict = {"gs_uri" : gsutil_output}
manifesto_df = pd.DataFrame(manifesto_dict, columns = ["gs_uri"])

manifesto_df

,gs_uri
0,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
1,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
2,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
3,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
4,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
5,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
6,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
7,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
8,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....
9,gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1....


In [ ]:
# generate a text file, which will be parsed by gsutil to download the selected patients:
manifesto_df.to_csv("gcs_paths.txt", header = False, index = False)

# check everything went as expected
!head /content/gcs_paths.txt

gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.100645791623978861750505647044382842552/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.100855571832074152951605738408734618579/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.101385144304667221218962368127740145808/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.10144581065043339122451692100395007780/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102719233427745502116416854281613435360/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102758679895916452229843467594324423877/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.102978961983855858543476361768905978287/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.10331314816593639915006005461010274587/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.103863628921590543632414419591397821320/
gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.32722.99.99.1043242987527

---

#### DICOM CT Series Download

In [ ]:
# if everything is allright, proceed with the download
!mkdir -p nsclc-radiomics

!cat gcs_paths.txt | gsutil -u idc-sandbox-000 -m cp -Ir ./nsclc-radiomics

In [48]:
data_base_path = "./nsclc-radiomics"

dataset_csv_name = 'nsclc-radiomics_bigquery.csv'
dataset_csv_path = os.path.join("/mnt/gdrive/My Drive/Colab Notebooks/data",
                                dataset_csv_name)

cohort_df = pd.read_csv(dataset_csv_path)

cohort_df.head()

,PatientID,ctStudyInstanceUID,ctSeriesInstanceUID,rtstructStudyInstanceUID,rtstructSeriesInstanceUID,segStudyInstanceUID,segSeriesInstanceUID
0,LUNG1-001,1.3.6.1.4.1.32722.99.99.2393413539117143687725...,1.3.6.1.4.1.32722.99.99.2989917765213423750108...,1.3.6.1.4.1.40744.29.2393413539117143687725971...,1.3.6.1.4.1.40744.29.2279381215866080725084441...,1.3.6.1.4.1.40744.29.2393413539117143687725971...,1.2.276.0.7230010.3.1.3.8323329.16296.15548365...
1,LUNG1-002,1.3.6.1.4.1.32722.99.99.2037150038059966416957...,1.3.6.1.4.1.32722.99.99.2329880015517990803358...,1.3.6.1.4.1.40744.29.2037150038059966416957653...,1.3.6.1.4.1.40744.29.2432675512669112458302594...,1.3.6.1.4.1.40744.29.2037150038059966416957653...,1.2.276.0.7230010.3.1.3.8323329.21133.15548295...
2,LUNG1-003,1.3.6.1.4.1.32722.99.99.2477262867958601216867...,1.3.6.1.4.1.32722.99.99.2389222799296192439904...,1.3.6.1.4.1.40744.29.2477262867958601216867965...,1.3.6.1.4.1.40744.29.2175894477461117410564218...,1.3.6.1.4.1.40744.29.2477262867958601216867965...,1.2.276.0.7230010.3.1.3.8323329.20958.15548278...
3,LUNG1-004,1.3.6.1.4.1.32722.99.99.2026036697036260886779...,1.3.6.1.4.1.32722.99.99.2809816144625926346520...,1.3.6.1.4.1.40744.29.2026036697036260886779831...,1.3.6.1.4.1.40744.29.1989243449739101957480841...,1.3.6.1.4.1.40744.29.2026036697036260886779831...,1.2.276.0.7230010.3.1.3.8323329.31481.15548267...
4,LUNG1-005,1.3.6.1.4.1.32722.99.99.7196186628043392557101...,1.3.6.1.4.1.32722.99.99.3490584753983772067630...,1.3.6.1.4.1.40744.29.7196186628043392557101987...,1.3.6.1.4.1.40744.29.1186162681418618501244963...,1.3.6.1.4.1.40744.29.7196186628043392557101987...,1.2.276.0.7230010.3.1.3.8323329.7097.155482712...


---

#### DICOM RTSTRUCT and RTSEG download

Due to how the TCIA dataset is organised, we need to do a bit of stitching to download the RTSTRUCTs/RTSEGs corresponding to the downloaded CTs.

In [66]:
os.listdir(data_base_path)

['1.3.6.1.4.1.32722.99.99.101385144304667221218962368127740145808',
 '1.3.6.1.4.1.32722.99.99.102719233427745502116416854281613435360',
 '1.3.6.1.4.1.32722.99.99.10331314816593639915006005461010274587',
 '1.3.6.1.4.1.32722.99.99.103863628921590543632414419591397821320',
 '1.3.6.1.4.1.32722.99.99.102978961983855858543476361768905978287',
 '1.3.6.1.4.1.32722.99.99.100645791623978861750505647044382842552',
 '1.3.6.1.4.1.32722.99.99.102758679895916452229843467594324423877',
 '1.3.6.1.4.1.32722.99.99.100855571832074152951605738408734618579',
 '1.3.6.1.4.1.32722.99.99.104324298752786706678094213253380791609',
 '1.3.6.1.4.1.32722.99.99.10144581065043339122451692100395007780']

In [67]:
rt_gsutil_uri_list = list()

for pat_StudyInstanceUID in os.listdir(data_base_path):

  path_to_study_folder = os.path.join(data_base_path, pat_StudyInstanceUID)

  pat_df = cohort_df[cohort_df["ctStudyInstanceUID"] == pat_StudyInstanceUID]

  pat_ctStudyInstanceUID = pat_df["ctStudyInstanceUID"].values[0]
  pat_rtstructStudyInstanceUID = pat_df["rtstructStudyInstanceUID"].values[0]
  pat_segStudyInstanceUID = pat_df["segStudyInstanceUID"].values[0]

  path_to_ct_folder = os.path.join(path_to_study_folder, pat_ctStudyInstanceUID)

  # both DICOM RTSTRUCT and RTSEG should be found in the same folder
  # path_to_rt_folder = path_to_seg_folder
  path_to_rt_folder = os.path.join(path_to_study_folder, pat_rtstructStudyInstanceUID)

  gsutil_uri = !gsutil -u idc-sandbox-000 ls gs://idc-tcia-1-nsclc-radiomics/dicom | grep $pat_rtstructStudyInstanceUID

  rt_gsutil_uri_list.append(gsutil_uri)



In [65]:
!gsutil -u idc-sandbox-000 ls gs://idc-tcia-1-nsclc-radiomics/dicom | grep $pat_rtstructStudyInstanceUID

gs://idc-tcia-1-nsclc-radiomics/dicom/1.3.6.1.4.1.40744.29.101385144304667221218962368127740145808/


---

Check the disk space after the download:

In [58]:

sitk_reader = sitk.ImageSeriesReader()
sitk_writer = sitk.ImageFileWriter()

series_ids = sitk_reader.GetGDCMSeriesIDs(path_to_series_folder)

path_to_series_folder

'./nsclc-radiomics/1.3.6.1.4.1.32722.99.99.101385144304667221218962368127740145808'

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   31G   72G  31% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.9G     0  5.9G   0% /dev/shm
tmpfs           6.4G   12K  6.4G   1% /var/colab
/dev/sda1       114G   33G   82G  29% /etc/hosts
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware


In [ ]:
!du -h ./nsclc-radiomics -d 0

599M	./nsclc-radiomics


---

## Python Preprocessing Pipeline Test

---

### WIP (SITK Pre-processing Functions)

In [ ]:
def sitk_resample(sitk_data, sitk_interpolator, output_spacing):
    
  orig_spacing = sitk_data.GetSpacing()
  orig_size = sitk_data.GetSize()

  new_size = [int(orig_size[0] * orig_spacing[0] / output_spacing[0]), 
              int(orig_size[1] * orig_spacing[1] / output_spacing[1]), 
              int(orig_size[2] * orig_spacing[2] / output_spacing[2])]

  res_filter = sitk.ResampleImageFilter()
  
  """
  arguments accepted by ResampleImageFilter():
  - image data
  - size
  - transform
  - interpolator
  - outputOrigin
  - outputSpacing
  - outputDirection
  - defaultPixelValue
  - outputPixelType
  """ 
    
  proc_sitk_data = res_filter.Execute(sitk_data,  
                                      new_size,  
                                      sitk.Transform(), 
                                      sitk_interpolator,  
                                      sitk_data.GetOrigin(), 
                                      output_spacing, 
                                      sitk_data.GetDirection(), 
                                      0, 
                                      sitk_data.GetPixelIDValue())

  return proc_sitk_data

In [ ]:
def dcm_ct_to_nrrd(path_to_series_folder, output_file_path,
                   resample_data = False, output_spacing = [1, 1, 1], interp = 'linear',
                   compress_data = True):
        
  sitk_reader = sitk.ImageSeriesReader()
  sitk_writer = sitk.ImageFileWriter()

  series_ids = sitk_reader.GetGDCMSeriesIDs(path_to_series_folder)

  # FIXME: only one series per folder is supported: generalise this?
  if not len(series_ids) == 1:
    print("ERROR: this function supports only one sequence per folder.")
    return 0

  if not interp in ["linear", "nn"]:
    print("Interpolation method unknown: falling back to 'linear'")
    interp = 'linear'

  elif interp == 'linear':
    sitk_interpolator = sitk.sitkLinear
  elif interp == 'nn':
    sitk_interpolator = sitk.sitkNearestNeighbor

  file_list = sitk_reader.GetGDCMSeriesFileNames(path_to_series_folder,
                                                  series_ids[0])
  sitk_reader.SetFileNames(file_list)
  sitk_vol = sitk_reader.Execute()


  if resample_data:
    sitk_vol = sitk_resample(sitk_data = sitk_vol,
                             sitk_interpolator = sitk_interpolator,
                             output_spacing = output_spacing)

  if compress_data:
    sitk_writer.UseCompressionOn()

  sitk_writer.SetFileName(output_path)
  sitk_writer.Execute(sitk_vol)

---

### Custom Library Functions

Custom library functions - should be good as they are.

In [54]:
"""
    ----------------------------------------
    IDC Radiomics use case (Colab Demo)
    
    useful functions for data handling/proc
    ----------------------------------------
    
    ----------------------------------------
    Author: Dennis Bontempi
    Email:  dennis_bontempi@dfci.harvard.edu
    Modified: 01 OCT 20
    ----------------------------------------
    
"""

## ----------------------------------------

# normalise the values of the volume between new_min_val and new_max_val
def normalise_volume(input_volume, new_min_val, new_max_val, old_min_val = None, old_max_val = None):

  """
  Normalise a numpy volume intensity in a range between two given values
  
  @params:
    input_volume - required : numpy volume to rescale (intensity-wise) in the new range.
    new_min_val  - required : the lower bound of the new intensity range.
    new_max_val  - required : the upper bound of the new intensity range.
    old_min_val  - optional : the lower bound of the old intensity range. Defaults to input_volume's np.min()
    old_max_val  - optional : the lower bound of the old intensity range. Defaults to input_volume's np.max()
    
  """
  
  # make sure the input volume is treated as a float volume
  input_volume = input_volume.astype(dtype = np.float16)

  # if no old_min_val and/or old_max_val are specified, default to the np.min() and np.max() of input_volume
  
  curr_min = np.min(input_volume) if old_min_val == None else old_min_val
  curr_max = np.max(input_volume) if old_max_val == None else old_max_val


  # normalise the values of each voxel between zero and one
  zero_to_one_norm = (input_volume - curr_min)/(curr_max - curr_min)

  # normalise between new_min_val and new_max_val
  return (new_max_val - new_min_val)*zero_to_one_norm + new_min_val

## ----------------------------------------
## ----------------------------------------

def compute_center_of_mass(input_mask):
  
  """
  Compute the center of mass (CoM) of a given binary 3D numpy segmask (fast version, numpy based).
  The idea is the following:
    - create a 4D vector starting from the 3D, populating each channel as follows:
      (x pos, y pos, z pos, mask_value);
    - keep only the nonzero values;
    - compute an average of the position (CoM) using such triplets (x, y, z).
  
  @params:
    input_mask - required : the 3D numpy vector (binary mask) to compute the center of mass of.
    
  @returns:
    ctr_mass - a list of three elements storing the coordinates of the CoM
               (the axes are the same as the input mask)
    
  """
  
  # sanity check: mask should be binary (multi-class RT is taken care of during in the export function)
  assert(len(np.unique(input_mask)) <= 2)
  
  # display and log warning
  if len(np.unique(input_mask)) == 1:
    print('WARNING: DICOM RTSTRUCT is empty.')
    return [-1, -1, -1]
  
  # clip mask values between 0 and 1 (e.g., to cope for masks with max val = 255)
  input_mask = np.clip(input_mask, a_min = 0, a_max = 1)
  
  segmask_4d = np.zeros(input_mask.shape + (4, ))

  # create a triplet of grids that will serve as axis for the next step
  y, x, z = np.meshgrid(np.arange(input_mask.shape[1]), 
                        np.arange(input_mask.shape[0]), 
                        np.arange(input_mask.shape[2]))
  
  # populate
  segmask_4d[..., 0] = x 
  segmask_4d[..., 1] = y 
  segmask_4d[..., 2] = z 
  segmask_4d[..., 3] = input_mask
  
  # keep only the voxels belonging to the mask
  nonzero_voxels = segmask_4d[np.nonzero(segmask_4d[:, :, :, 3])]

  # average the (x, y, z) triplets
  ctr_mass = np.average(nonzero_voxels[:, :3], axis = 0)
    
  return ctr_mass

  
## ----------------------------------------
## ----------------------------------------

def get_bbox_dict(int_center_of_mass, seg_mask_shape, bbox_size, z_first = True):

  """
  Compute the crop bounding box indices starting from the CoM and the BBox size.
  
  @params:
    int_center_of_mass - required : a list of integers (int(CoM)) where CoM is the output
                                    of the function "compute_center_of_mass()"
    bbox_size          - required : the size of the bounding box along each axis.
    z_first            - optional : if True, returns the dict as (lon, cor, sag), i.e., (z, y, x),
                                    as opposed to (sag, cor, lon), i.e., (x, y, z) (defaults to True).
  
  @returns:
    bbox : a dictionary formatted as follows:
      {
       'lon': {'first': 89, 'last': 139}
       'cor': {'first': 241, 'last': 291},
       'sag': {'first': 150, 'last': 200}
      }
  """
  
  assert len(int_center_of_mass) == 3
  assert len(bbox_size) == 3
    
  bbox = dict()

  bbox['cor'] = dict()
  bbox['sag'] = dict()
  bbox['lon'] = dict()

  # take care of axes shuffling 
  cor_idx = 1
  lon_idx = 0 if z_first else 2
  sag_idx = 2 if z_first else 0
  
  
  # bounding box if no exception is found
  sag_first = int(int_center_of_mass[sag_idx] - np.round(bbox_size[sag_idx]/2))
  sag_last = int(int_center_of_mass[sag_idx] + np.round(bbox_size[sag_idx]/2)) - 1
  
  cor_first = int(int_center_of_mass[cor_idx] - np.round(bbox_size[cor_idx]/2))
  cor_last = int(int_center_of_mass[cor_idx] + np.round(bbox_size[cor_idx]/2)) - 1
  
  lon_first = int(int_center_of_mass[lon_idx] - np.round(bbox_size[lon_idx]/2))
  lon_last = int(int_center_of_mass[lon_idx] + np.round(bbox_size[lon_idx]/2)) - 1
  

  # print out exceptions
  if sag_last > seg_mask_shape[sag_idx] - 1 or sag_first < 0:
    print('WARNING: the bounding box size exceeds volume dimensions (sag axis)')
    print('Cropping will be performed ignoring the "bbox_size" parameter')
    
  if cor_last > seg_mask_shape[cor_idx] - 1 or cor_first < 0:
    print('WARNING: the bounding box size exceeds volume dimensions (cor axis)')
    print('Cropping will be performed ignoring the "bbox_size" parameter')
    
  if lon_last > seg_mask_shape[lon_idx] - 1 or lon_first < 0:
    print('WARNING: the bounding box size exceeds volume dimensions (lon axis)')
    print('Cropping will be performed ignoring the "bbox_size" parameter')
    
    
  # take care of exceptions where bbox is bigger than the actual volume
  sag_first = int(np.max([0, sag_first]))
  sag_last = int(np.min([seg_mask_shape[sag_idx] - 1, sag_last]))
  
  cor_first = int(np.max([0, cor_first]))
  cor_last = int(np.min([seg_mask_shape[cor_idx] - 1, cor_last]))
  
  lon_first = int(np.max([0, lon_first]))
  lon_last = int(np.min([seg_mask_shape[lon_idx] - 1, lon_last]))
  
  
  # populate the dictionary and return it
  bbox['sag']['first'] = sag_first
  bbox['sag']['last'] = sag_last

  bbox['cor']['first'] = cor_first
  bbox['cor']['last'] = cor_last

  bbox['lon']['first'] = lon_first
  bbox['lon']['last'] = lon_last
    
  return bbox
  

## ----------------------------------------
## ----------------------------------------

def get_input_volume(input_ct_nrrd_path):

  """
  Should accept path to (ideally 150x150x150) NRRD volumes only, obtained
  exploting the pipeline in "lung1_preprocessing.ipynb" using the functions above.
  
  FIXME: add as params also "com_int" and "final_crop_size = (50, 50, 50)"
         (handling of volumes that are not 150x150x150?)
  
  """
  
  sitk_ct_nrdd = sitk.ReadImage(input_ct_nrrd_path)
  ct_nrdd = sitk.GetArrayFromImage(sitk_ct_nrdd)
      
  # volume intensity normalisation, as seen in:
  # https://github.com/modelhub-ai/deep-prognosis/blob/master/contrib_src/processing.py
  ct_nrdd_norm = normalise_volume(input_volume = ct_nrdd,
                                  new_min_val = 0,
                                  new_max_val = 1,
                                  old_min_val = -1024,
                                  old_max_val = 3071)
    
  # FIXME: handle exceptions for volumes that are not 150x150x150
  ct_nrdd_norm_crop = ct_nrdd_norm[50:100, 50:100, 50:100]
  
  """
  # FIXME: debug
  print(ct_nrdd.shape)
  print(ct_nrdd_norm.shape)
  print(min_x, max_x)
  print(min_y, max_y)
  print(min_z, max_z)
  print(ct_nrdd_norm_crop.shape)
  """
  
  return ct_nrdd_norm_crop

---

### To Re-write (Plastimatch to SITK)

Old scripts, exploiting Plastimatch. Re-write using SITK functions.

In [ ]:
def export_com_subvolume(ct_nrrd_path, rt_nrrd_path, crop_size, output_dir, subj_id,
                         z_first = True, rm_orig = False):

  """
  Main reason: save space
  
  Use plastimatch so that we don't need to load and then write NRRD through python (pynrrd)
  
  """
  
  # sanity check
  assert(os.path.exists(ct_nrrd_path))
  assert(os.path.exists(rt_nrrd_path))
  
  sitk_seg = sitk.ReadImage(rt_nrrd_path)
  seg = sitk.GetArrayFromImage(sitk_seg)
  
  # output dictionary contains info regarding CoM and the cropping op.s
  out_log = dict()
    
  com = compute_center_of_mass(input_mask = seg)
  com_int = [int(coord) for coord in com]
  
  out_log["com"] = com
  out_log["com_int"] = com_int
  
  # if CoM calculation goes wrong, abort returning the out_log so far
  if sum(com_int) < 0:
    print('WARNING: CoM calculation resulted in an error, aborting... ')
    return out_log
  
  # otherwise go on with the processing and the cropping
  bbox = get_bbox_dict(com_int, seg_mask_shape = seg.shape, bbox_size = crop_size)
  
  # make sure no bounding box exceeds the dimension of the volume
  # (should be taken care of already in the get_bbox_dict() function)
  cor_idx = 1
  lon_idx = 0 if z_first else 2
  sag_idx = 2 if z_first else 0 
  
  # less and not leq at the second term --> the last slice of the volume is seg.shape[...] - 1
  assert(bbox['sag']['first'] >= 0 and bbox['sag']['last'] < seg.shape[sag_idx])
  assert(bbox['cor']['first'] >= 0 and bbox['cor']['last'] < seg.shape[cor_idx])
  assert(bbox['lon']['first'] >= 0 and bbox['lon']['last'] < seg.shape[lon_idx])
  
  
  # cropped nrrd files path
  ct_nrrd_crop_path = os.path.join(output_dir, subj_id + '_ct_res_crop.nrrd')
  rt_nrrd_crop_path = os.path.join(output_dir, subj_id + '_rt_res_crop.nrrd')
  
  if z_first:
    xmin = str(bbox["sag"]["first"]); xmax = str(bbox["sag"]["last"])
    ymin = str(bbox["cor"]["first"]); ymax = str(bbox["cor"]["last"])
    zmin = str(bbox["lon"]["first"]); zmax = str(bbox["lon"]["last"])
  else:
    xmin = str(bbox["lon"]["first"]); xmax = str(bbox["lon"]["last"])
    ymin = str(bbox["cor"]["first"]); ymax = str(bbox["cor"]["last"])
    zmin = str(bbox["sag"]["first"]); zmax = str(bbox["sag"]["last"])
    
  
  # crop the NRRD CT file to the crop_size subvolume
  bash_command = list()
  bash_command += ["plastimatch", "crop"]
  bash_command += ["--input", ct_nrrd_path]
  bash_command += ["--output", ct_nrrd_crop_path]
  bash_command += ["--voxels", "%s %s %s %s %s %s"%(xmin, xmax, ymin, ymax, zmin, zmax)]

  # print progress info
  print("\nCropping the resampled NRRD CT to bbox using plastimatch... ", end = '')
  out_log['dcm_nrrd_ct_cropping'] = subprocess.call(bash_command)
  print("Done.")
  
  
  # crop the NRRD RT file to the crop_size subvolume
  bash_command = list()
  bash_command += ["plastimatch", "crop"]
  bash_command += ["--input", rt_nrrd_path]
  bash_command += ["--output", rt_nrrd_crop_path]
  bash_command += ["--voxels", "%s %s %s %s %s %s"%(xmin, xmax, ymin, ymax, zmin, zmax)]
    
  # print progress info
  print("Cropping the resampled NRRD RTSTRUCT to bbox using plastimatch... ", end = '')
  out_log['dcm_nrrd_rt_cropping'] = subprocess.call(bash_command)
  print("Done.")
  
  # log some useful information about the cropping
  log_file_path = os.path.join(output_dir, subj_id + '_crop_log.json')
  with open(log_file_path, 'w') as json_file:
    json.dump(bbox, json_file, indent = 2)
  
  if rm_orig:
    # clean up
    print("\nRemoving the resampled NRRD files... ", end = '')
    os.remove(ct_nrrd_path)
    os.remove(rt_nrrd_path)
    print("Done.")
  
  return out_log

## ----------------------------------------
## ----------------------------------------

def export_res_nrrd_from_dicom(dicom_ct_path, dicom_rt_path, output_dir, subj_id,
                               ct_interpolation = 'linear', output_dtype = "int"):
  
  """
  Convert DICOM CT and RTSTRUCT sequences to NRRD files and resample to 1-mm isotropic
  exploiting plastimatch (direct call, bash-like).
  
  @params:
    dicom_ct_path - required :
    dicom_rt_path - required :
    output_dir    - required : 
    subj_id       - required :
    output_dtype  - optional : 
    
  @returns:
    out_log : 
    
  """
  
  out_log = dict()
  
  # temporary nrrd files path (DICOM to NRRD, no resampling)
  ct_nrrd_path = os.path.join(output_dir, 'tmp_ct_orig.nrrd')
  rt_folder = os.path.join(output_dir, subj_id  + '_whole_ct_rt')
  
  # log the labels of the exported segmasks
  rt_struct_list_path = os.path.join(output_dir, subj_id + '_rt_list.txt')
  
  # convert DICOM CT to NRRD file - no resampling
  bash_command = list()
  bash_command += ["plastimatch", "convert"]
  bash_command += ["--input", dicom_ct_path]
  bash_command += ["--output-img", ct_nrrd_path]
                   
  # print progress info
  print("Converting DICOM CT to NRRD using plastimatch... ", end = '')
  out_log['dcm_ct_to_nrrd'] = subprocess.call(bash_command)
  print("Done.")
  
  
  # convert DICOM RTSTRUCT to NRRD file - no resampling
  bash_command = list()
  bash_command += ["plastimatch", "convert"]
  bash_command += ["--input", dicom_rt_path]
  bash_command += ["--referenced-ct", dicom_ct_path]
  bash_command += ["--output-prefix", rt_folder]
  bash_command += ["--prefix-format", 'nrrd']
  bash_command += ["--output-ss-list", rt_struct_list_path]
  
  # print progress info
  print("Converting DICOM RTSTRUCT to NRRD using plastimatch... ", end = '')
  out_log['dcm_rt_to_nrrd'] = subprocess.call(bash_command)
  print("Done.")
  
  # look for the labelmap for GTV
  gtv_rt_file = [f for f in os.listdir(rt_folder) if 'gtv-1' in f.lower()][0]
  rt_nrrd_path = os.path.join(rt_folder, gtv_rt_file)
  
  ## ----------------------------------------
  
  # actual nrrd files path 
  res_ct_nrrd_path = os.path.join(output_dir, subj_id + '_ct_resampled.nrrd')
  res_rt_nrrd_path = os.path.join(output_dir, subj_id + '_rt_resampled.nrrd')
  
  # resample the NRRD CT file to 1mm isotropic
  bash_command = list()
  bash_command += ["plastimatch", "resample"]
  bash_command += ["--input", ct_nrrd_path]
  bash_command += ["--output", res_ct_nrrd_path]
  bash_command += ["--spacing", "1 1 1"]
  bash_command += ["--interpolation", ct_interpolation]
  bash_command += ["--output-type", output_dtype]
  
  # print progress info
  print("\nResampling NRRD CT to 1mm isotropic using plastimatch... ", end = '')
  out_log['dcm_nrrd_ct_resampling'] = subprocess.call(bash_command)
  print("Done.")
  
  # FIXME: log informations about the native volume
  #out_log["shape_original"] = list(tmp.)
  
  
  # resample the NRRD RTSTRUCT file to 1mm isotropic
  bash_command = list()
  bash_command += ["plastimatch", "resample"]
  bash_command += ["--input", rt_nrrd_path]
  bash_command += ["--output", res_rt_nrrd_path]
  bash_command += ["--spacing", "1 1 1"]
  bash_command += ["--interpolation", "nn"]
    
  # print progress info
  print("Resampling NRRD RTSTRUCT to 1mm isotropic using plastimatch... ", end = '')
  out_log['dcm_nrrd_rt_resampling'] = subprocess.call(bash_command)
  print("Done.")

  
  # clean up
  print("\nRemoving temporary files (DICOM to NRRD, non-resampled)... ", end = '')
  os.remove(ct_nrrd_path)
  # FIXME: keep the RTSTRUCTs (latest LUNG1 has multiple structures --> additional checks afterwards)?
  #os.remove(rt_nrrd_path)
  print("Done.")
  

  return out_log
